In [46]:
# Importing packages

import pandas as pd
import numpy as np
import plotnine as p9

In [47]:
# Setting routes for reading in csv data

data_masterplan_input_target = 'test_mp_data.csv'
phasing_logic_input_target = 'phasing_logic_matrix.csv'

# Reading in masterplan data and phasing logic matrix

data_masterplan_input =  pd.read_csv(data_masterplan_input_target)
phasing_logic_input = pd.read_csv(phasing_logic_input_target).rename({'delivery_status':'Phasing_Note'},axis = 1)


In [48]:
# Dropnas from the project ID column from the main data_masterplan_input data frame

data_masterplan_input = data_masterplan_input.dropna(subset = ['ProjectID'],axis = 0).reset_index(drop = True)

In [49]:
# Find columns with a '%' sign and the column immediately to the left in the main masterplan input dataframe

columns_with_percent = [col for col in data_masterplan_input.columns if '%' in col]

selected_columns = []

for col in columns_with_percent:

    # Find the index of the column with a '%' sign
    
    col_index = data_masterplan_input.columns.get_loc(col)
    # Append the column to the left if it exists
    
    if col_index > 0:
        selected_columns.append(data_masterplan_input.columns[col_index - 1])
    
    # Append the column with a '%' sign
    
    selected_columns.append(col)

# Select the identified columns from the DataFrame
committed_development = data_masterplan_input[selected_columns]
committed_development = pd.concat([data_masterplan_input[['ProjectID','Phasing_Note','Approved Completion Year']],committed_development],axis = 1)

In [50]:
# Setting start dates of the plan

plan_start = 2024
plan_end = 2040

# Creating sequence containing all years of the plan phasing period

phasing_period = np.arange(plan_start,plan_end+1,1)
phasing_period = phasing_period.astype(str)
print(phasing_period)


['2024' '2025' '2026' '2027' '2028' '2029' '2030' '2031' '2032' '2033'
 '2034' '2035' '2036' '2037' '2038' '2039' '2040']


In [51]:
# Creating dataframe includes the manual phasing overrides

phasing_override_input = data_masterplan_input[phasing_period]
phasing_override_input = pd.concat([data_masterplan_input['ProjectID'],phasing_override_input],axis = 1)

# Creating data frame with phasing timeline for attaching to different development phasing pipelines

timeline = phasing_override_input

In [52]:
# Identifying which development group committed development should be entered in based on delivery status

committed_development =committed_development.merge(phasing_logic_input,how = 'left',on = 'Phasing_Note')

# Dropping masterplans without a status or are complete

committed_development= committed_development.dropna(subset=['group'],axis =0)
committed_development = committed_development[committed_development['group'] != 'Remove'].reset_index(drop = True)


In [53]:
# Creating a data frame that contains under construction development
under_construction_development = committed_development[committed_development['group'] == 'Under Construction'].reset_index(drop = True)
under_construction_development['Approved Completion Year'] =under_construction_development['Approved Completion Year'].apply(lambda x: 1 if pd.isna(x) or not isinstance(x, int) else x
).astype(int)

# Removing un-needed columns
under_construction_development = under_construction_development.drop(['group','delay_period_yrs','Phasing_Note'],axis = 1)

# Melting the data frame into long form
under_construction_development = under_construction_development.melt(id_vars= ['ProjectID','Approved Completion Year'])

# Splitting the long-form data frame into two parts and merging those parts together - the purpose being to split the % committed and number of units associated with each land use into separatre columns
under_construction_development_split_1 = under_construction_development[~under_construction_development['variable'].str.contains('%')].reset_index(drop = True).rename({'variable':'land_use','value':'number_units'},axis = 1)
under_construction_development_split_2 = under_construction_development[under_construction_development['variable'].str.contains('%')].reset_index(drop = True).drop(['ProjectID','variable','Approved Completion Year'],axis = 1).rename({'value':'per_committed'},axis = 1)
under_construction_development = pd.concat([under_construction_development_split_1,under_construction_development_split_2],axis = 1)

# Creating starting and end of phasing year columns

under_construction_development['start_year'] = plan_start
under_construction_development['end_year'] = under_construction_development['Approved Completion Year'].apply(lambda x: (plan_start + 1) if pd.isna(x) or x <= plan_start else x)

# Calculating the percentage of development over the course of each phasing period

under_construction_development['development_pa'] = under_construction_development['per_committed']/(under_construction_development['end_year'] - under_construction_development['start_year'])



In [54]:
# Merging data and preprocessing
under_construction_development_phased = under_construction_development.merge(timeline, how='left', on='ProjectID')
under_construction_development_phased = under_construction_development_phased.drop(['Approved Completion Year', 'number_units', 'per_committed'], axis=1)
under_construction_development_phased = under_construction_development_phased.melt(id_vars=['ProjectID', 'land_use', 'start_year', 'end_year', 'development_pa'])

# Ensuring that datatype conversions are handled for all related columns
under_construction_development_phased['variable'] = pd.to_numeric(under_construction_development_phased['variable'], errors='coerce')  # Converting the 'variable' which probably holds year values
under_construction_development_phased['start_year'] = pd.to_numeric(under_construction_development_phased['start_year'], errors='coerce')  # Converting 'start_year' to numeric if it isn't already
under_construction_development_phased['end_year'] = pd.to_numeric(under_construction_development_phased['end_year'], errors='coerce')  # Converting 'end_year' to numeric
under_construction_development_phased['development_pa'] = pd.to_numeric(under_construction_development_phased['development_pa'], errors='coerce')  # Converting 'development_pa'

# Handling the conditional logic to determine the value based on year range
under_construction_development_phased['value'] = under_construction_development_phased.apply(
    lambda row: row['development_pa'] if (row['variable'] >= row['start_year'] and row['variable'] <= row['end_year']) else 0, 
    axis=1)

under_construction_development_phased = under_construction_development_phased[['ProjectID','land_use','variable','value']].rename({'variable':'year','value':'development_pa'},axis =1)
under_construction_development_phased['status'] = 'Under Construction'

In [55]:
# Creating a data frame that contains under construction development
pipeline_development = committed_development[committed_development['group'] == 'Pipeline'].reset_index(drop = True)
pipeline_development['Approved Completion Year'] = pipeline_development['Approved Completion Year'].apply(lambda x: 1 if pd.isna(x) or not isinstance(x, int) else x
).astype(int)

# Removing un-needed columns
pipeline_development= pipeline_development.drop(['group','Phasing_Note'],axis = 1)

# Melting the data frame into long form
pipeline_development= pipeline_development.melt(id_vars= ['ProjectID','Approved Completion Year','delay_period_yrs'])

# Splitting the long-form data frame into two parts and merging those parts together - the purpose being to split the % committed and number of units associated with each land use into separatre columns
pipeline_development_split_1 = pipeline_development[~pipeline_development['variable'].str.contains('%')].reset_index(drop = True).rename({'variable':'land_use','value':'number_units'},axis = 1)
pipeline_development_split_2 = pipeline_development[pipeline_development['variable'].str.contains('%')].reset_index(drop = True).drop(['ProjectID','variable','Approved Completion Year','delay_period_yrs'],axis = 1).rename({'value':'per_committed'},axis = 1)
pipeline_development = pd.concat([pipeline_development_split_1,pipeline_development_split_2],axis = 1)
pipeline_development['delay_period_yrs'] = pipeline_development['delay_period_yrs'].astype(int)

# Creating starting and end of phasing year columns

pipeline_development['start_year'] =  pipeline_development['delay_period_yrs'] + plan_start
pipeline_development['end_year'] = pipeline_development['Approved Completion Year'].apply(lambda x: plan_end if pd.isna(x) or x <= plan_start or x < plan_end else x)

# Calculating the percentage of development over the course of each phasing period
pipeline_development['development_pa'] = pipeline_development['per_committed']/(pipeline_development['end_year'] - pipeline_development['start_year'])


In [56]:
# Merging data and preprocessing
pipeline_development_phased = pipeline_development.merge(timeline, how='left', on='ProjectID')
pipeline_development_phased = pipeline_development_phased.drop(['Approved Completion Year', 'number_units', 'per_committed','delay_period_yrs'], axis=1)
pipeline_development_phased = pipeline_development_phased.melt(id_vars=['ProjectID', 'land_use', 'start_year', 'end_year', 'development_pa'])

# Ensuring that datatype conversions are handled for all related columns
pipeline_development_phased['variable'] = pd.to_numeric(pipeline_development_phased['variable'], errors='coerce')  # Converting the 'variable' which probably holds year values
pipeline_development_phased['start_year'] = pd.to_numeric(pipeline_development_phased['start_year'], errors='coerce')  # Converting 'start_year' to numeric if it isn't already
pipeline_development_phased['end_year'] = pd.to_numeric(pipeline_development_phased['end_year'], errors='coerce')  # Converting 'end_year' to numeric
pipeline_development_phased['development_pa'] = pd.to_numeric(pipeline_development_phased['development_pa'], errors='coerce')  # Converting 'development_pa'

# Handling the conditional logic to determine the value based on year range
pipeline_development_phased['value'] = pipeline_development_phased.apply(
    lambda row: row['development_pa'] if (row['variable'] >= row['start_year'] and row['variable'] <= row['end_year']) else 0, 
    axis=1)

pipeline_development_phased = pipeline_development_phased[['ProjectID','land_use','variable','value']].rename({'variable':'year','value':'development_pa'},axis =1)
pipeline_development_phased['status'] = 'Pipeline'

In [57]:
# Collating under construction and pipeline development into one section

development_phased = pd.concat([under_construction_development_phased,pipeline_development_phased],axis = 0).reset_index(drop = True)

In [61]:
override_phased = phasing_override_input.melt(id_vars= 'ProjectID').reset_index(drop = True)
override_phased['value'] = override_phased.apply(lambda row: 0.05 if row['ProjectID'] == 'Riyadh' else row['value'],axis = 1)
override_phased = override_phased.rename({'variable':'year','value':'override_development_pa'},axis =1)
override_phased

,ProjectID,year,override_development_pa
0,Riyadh,2024,0.05
1,Samha New Housing - Ph 3,2024,NaN
2,Samha New Housing - Ph 2,2024,NaN
3,ABU MREIKHAH_2,2024,NaN
4,26626.0,2024,NaN
...,...,...,...
4636,GHANADHAH_2,2040,NaN
4637,1113.13,2040,NaN
4638,Al Faya Industrial - New,2040,NaN
4639,32379.0,2040,NaN


In [59]:
development_phased

,ProjectID,land_use,year,development_pa,status
0,Riyadh,Residential - Villas Emiratis (Units),2024,1.000000,Under Construction
1,1099.0,Residential - Villas Emiratis (Units),2024,0.000000,Under Construction
2,2100.0,Residential - Villas Emiratis (Units),2024,1.000000,Under Construction
3,1170.1,Residential - Villas Emiratis (Units),2024,1.000000,Under Construction
4,5000.0,Residential - Villas Emiratis (Units),2024,1.000000,Under Construction
...,...,...,...,...,...
40183,10234.0,Other (GFA),2040,0.000000,Pipeline
40184,AL SMEEH_21,Other (GFA),2040,0.000000,Pipeline
40185,GHANADHAH_2,Other (GFA),2040,0.000000,Pipeline
40186,1113.13,Other (GFA),2040,0.090909,Pipeline
